<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Thu Dec  8 17:29:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [55]:
!pip3 install transformers
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [57]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [58]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [59]:
device

'cuda'

# Lecture et analyse pandas

In [60]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

In [61]:
df = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [62]:
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

In [63]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    #text = REPLACE_BY_SPACE_RE.sub(' ', text)
    #text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
df['data'] = df['data'].apply(clean_text)
#df['data'] = df['data'].str.replace('\d+', '')

In [65]:
df.head()

,data,label
0,22e jour consécutif grève reconductible contre...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"« n’y a risque pénurie » carburant, a déclaré ...",0


In [66]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//400 >0:
        n = len(text1.split())//400
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:450]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*400:w*400 + 450]
            l_total.append(" ".join(l_parcial))
    return l_total

In [67]:
df['text_split'] = df['data'].apply(get_split)

In [68]:
for index, row in df.iterrows():
  if len(row['text_split']) > 1: 
    print(index)
    break

31


In [69]:
print(df['text_split'][31][0].split()[400:])
print(df['text_split'][31][1].split()[:100])

['océans', 'avant', 'après', 'crise', 'kt.', 'chercheur', 'northwestern', 'university', 'coquilles', 'fossilisée', 'collectée', 'lors', "d'un", 'voyage', 'recherche', 'terrain', "l'île", 'seymour,', 'antarctique.', '©', 'northwestern', 'university', 'effet', 'serre', 'acidification', 'océans', 'informations', 'fournies', 'analyses', 'conduites', 'méthode', 'spectrométrie', 'masse', ',', 'similaire', 'celles', 'utilisées', 'membres', 'lsce', 'france', 'locaux', 'ice', ',', 'permis', "d'établir", 'océans', 'subissaient', 'acidification', 'rapide', 'avant']
['océans', 'avant', 'après', 'crise', 'kt.', 'chercheur', 'northwestern', 'university', 'coquilles', 'fossilisée', 'collectée', 'lors', "d'un", 'voyage', 'recherche', 'terrain', "l'île", 'seymour,', 'antarctique.', '©', 'northwestern', 'university', 'effet', 'serre', 'acidification', 'océans', 'informations', 'fournies', 'analyses', 'conduites', 'méthode', 'spectrométrie', 'masse', ',', 'similaire', 'celles', 'utilisées', 'membres', 'l

In [70]:
train_l = []
label_l = []
index_l =[]
for idx,row in df.iterrows():
    for l in row['text_split']:
        train_l.append(l)
        label_l.append(row['label'])
        index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(1568, 1568, 1568)

In [71]:
df = pd.DataFrame({'data':train_l, 'label':label_l})
df.head()

,data,label
0,22e jour consécutif grève reconductible contre...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"« n’y a risque pénurie » carburant, a déclaré ...",0


In [72]:
new_df=df

df_target=pd.DataFrame(new_df['label'])
df_target['temp']=0
for i in range(len(df_target)):
    if df_target['label'].loc[i]==0:
        df_target['temp'].loc[i]=1
new_df1=new_df.drop(['label'],axis=1)
new_df2=pd.DataFrame(df_target)

new_df3=pd.concat([new_df1,new_df2],axis = 1)
df=new_df3

df['label'] = df[df.columns[1:]].values.tolist()
new_df = df[['data', 'label']].copy()
df = new_df
df = df.rename(columns={'data': 'text'})
df.head()

,text,label
0,22e jour consécutif grève reconductible contre...,"[0, 1]"
1,"depuis plusieurs mois, initiatives chercheurs ...","[0, 1]"
2,google vient d'introduire mise jour applicatio...,"[0, 1]"
3,portrait. chacun s’empresse autour d’elle tand...,"[0, 1]"
4,"« n’y a risque pénurie » carburant, a déclaré ...","[0, 1]"


In [73]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [74]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 1
LEARNING_RATE = 1e-05

In [75]:
!pip install sentencepiece

tranfo_name = "cmarkea/distilcamembert-base"
tokenizer = AutoTokenizer.from_pretrained(tranfo_name)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 


Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream

In [76]:
df.head()

,text,label
0,22e jour consécutif grève reconductible contre...,"[0, 1]"
1,"depuis plusieurs mois, initiatives chercheurs ...","[0, 1]"
2,google vient d'introduire mise jour applicatio...,"[0, 1]"
3,portrait. chacun s’empresse autour d’elle tand...,"[0, 1]"
4,"« n’y a risque pénurie » carburant, a déclaré ...","[0, 1]"


In [77]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1568, 2)
TRAIN Dataset: (1254, 2)
TEST Dataset: (314, 2)


In [78]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [79]:
import torch

class BERTClass(torch.nn.Module):
    def __init__(self):
      super(BERTClass, self).__init__()
      self.l1 = CamembertModel.from_pretrained(tranfo_name)
      self.l3 = torch.nn.Linear(768, 2) #2 = binary classification
    
    def forward(self, ids, mask, token_type_ids):
      output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output = self.l3(output_1['pooler_output'])

      return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream

BERTClass(
  (l1): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [80]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [81]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        targets = data['targets'].to(device)
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [82]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.7161461114883423
Epoch: 0, Loss:  0.6103151440620422
Epoch: 0, Loss:  0.5934352278709412
Epoch: 0, Loss:  0.41235366463661194
Epoch: 0, Loss:  0.4262455999851227
Epoch: 0, Loss:  0.41789913177490234
Epoch: 0, Loss:  0.12241990864276886
Epoch: 0, Loss:  0.17462067306041718
Epoch: 0, Loss:  0.13300791382789612
Epoch: 0, Loss:  0.10301690548658371
Epoch: 0, Loss:  0.08463000506162643
Epoch: 0, Loss:  0.15574871003627777
Epoch: 0, Loss:  0.14918236434459686


In [86]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [87]:
from sklearn import metrics
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Accuracy Score = 0.9203821656050956
F1 Score (Micro) = 0.9203821656050956
F1 Score (Macro) = 0.9160921383143605


In [88]:
torch.save(model.state_dict(), "model.pth")